# <center>XGB for 5 X 5 GRID <center> 

## All Imports 

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.decomposition import PCA
import math
from numpy import mean
from numpy import std

## Reading and Transforming data

In [20]:
dataset = pd.read_csv("Table_5X5_Final.csv")
dataTypeSeries = dataset.dtypes
dataset = dataset.sample(frac=1, random_state=2)
print(dataTypeSeries)

OBJECTID                         int64
gridcode                         int64
Bouguer_MEAN                   float64
Fault_COUNT                      int64
Fault_MIN                      float64
                                ...   
Mean_Distance_Faults_STD       float64
Mean_Distance_QAFaults_MEAN    float64
Mean_Distance_QAFaults_STD     float64
Distance_QAFaults_MEAN         float64
Distance_QAFaults_STD          float64
Length: 248, dtype: object


In [21]:
selected_rows = dataset[~dataset["Grad_Geot"].isnull()]

In [22]:
X = (selected_rows.drop(["gridcode", "Shape_Length", "Shape_Area", "HFD", "Grad_Geot", "Count of Points"], axis=1))
Y = (selected_rows["Grad_Geot"])

In [23]:
Y_mean, Y_std = mean(Y), std(Y)
# identify outliers
cut_off = Y_std * 3
lower, upper = Y_mean - cut_off, Y_mean + cut_off
# identify outliers
outliers = [x for x in Y if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in Y if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))
print(outliers)

Identified outliers: 2
Non-outlier observations: 107
[48.0, 42.0]


In [52]:
selected_rows = dataset[~dataset["Grad_Geot"].isnull()]

In [53]:
FinalData = selected_rows[~selected_rows['Grad_Geot'].isin(outliers)]

In [54]:
X = pd.DataFrame((FinalData.drop(["gridcode", "Shape_Length", "Shape_Area", "HFD", "Grad_Geot", "Count of Points" ], axis=1)))
Y = (FinalData["Grad_Geot"])

In [55]:
Y.shape

(107,)

In [56]:
Y.describe()

count    107.000000
mean      21.572968
std        5.274973
min       10.448200
25%       17.381743
50%       20.600000
75%       24.512500
max       37.000000
Name: Grad_Geot, dtype: float64

## Model - Geothermal Gradient 

In [57]:
Regressor = XGBRegressor(gamma=10,
 learning_rate=0.1, max_depth=15, n_estimators=500, random_state=2, reg_lambda=10)
Regressor.fit(X, Y)
pred_Regressor = Regressor.predict(X)

## K fold CV

In [58]:
k = 5
kf = KFold(n_splits=k)
 
result = cross_val_score(Regressor , X, Y, cv = kf)
 
print("Avg accuracy: {}".format(result.mean()))

Avg accuracy: 0.14861505029999378


In [31]:
k = 5
kf = KFold(n_splits=k)
result = cross_val_score(Regressor , X, Y, cv = kf, scoring='neg_mean_absolute_error')
print (result.mean())

-3.7879589917757834


In [32]:
k = 5
kf = KFold(n_splits=k)
result = cross_val_score(Regressor , X, Y, cv = kf, scoring='neg_mean_squared_error')
print (result.mean())
rmse = math.sqrt(-result.mean())
rmse

-23.163680747112434


4.81286616758792

### Assessing features correlation and importances 

In [33]:
BestFeatures = SelectKBest(score_func=f_regression, k=110)
fit = BestFeatures.fit(X, Y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
FeatureScores = pd.concat([dfcolumns, dfscores], axis=1)
FeatureScores.columns = ['Variable', 'Scores']
print(FeatureScores.nlargest(100, 'Scores'))

C:\Users\mathe\anaconda3\envs\Teste_Tensor\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: divide by zero encountered in true_divide
  correlation_coefficient /= X_norms
C:\Users\mathe\anaconda3\envs\Teste_Tensor\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:289: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= X_norms
C:\Users\mathe\anaconda3\envs\Teste_Tensor\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:358: RuntimeWarning: invalid value encountered in true_divide
  f_statistic = corr_coef_squared / (1 - corr_coef_squared) * deg_of_freedom


                                              Variable     Scores
239                         Mean_Distance_QAFaults_STD  13.439664
234                                     Mean_Drain_STD  11.657323
227                                    Mean_Fault_MEAN  11.465879
59                                Soil_Siliceous_light  11.448007
230                                     Mean_Elevation  11.389316
..                                                 ...        ...
64                             ONEHOT_Sistema_C_mbrico   1.554974
138  MAX_ONEHOT_Descricao1_Cret_cico_Superior_da_Or...   1.539519
156  MAX_ONEHOT_Descricao1_Dep_sitos_paleog_nicos_d...   1.505654
142  MAX_ONEHOT_Descricao1_Dep_sitos_de_cobertura_d...   1.462202
57                             Soil_Intermediate_loamy   1.455201

[100 rows x 2 columns]


In [34]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(Regressor, random_state=1).fit(X, Y)
eli5.show_weights(perm, feature_names = X.columns.tolist(), top=40)

Weight,Feature
0.2135 ± 0.0663,Mean_Distance_QAFaults_STD
0.1264 ± 0.0284,Mean_Fault_STD
0.1057 ± 0.0594,MAX_ONEHOT_Descricao1_Dev_nico_do_Bordo_Sudoeste
0.0984 ± 0.0336,MAX_ONEHOT_Descricao1_Rochas_magm_ticas_intrusivas__granit_ides_rel1
0.0812 ± 0.0125,Mean_Drain_STD
0.0794 ± 0.0463,Distance_QAFaults_STD
0.0505 ± 0.0237,Bouguer_MEAN
0.0487 ± 0.0161,Fault_RANGE
0.0381 ± 0.0174,Mean_Elevation
0.0352 ± 0.0263,Temperature_MAX


## Grid Search CV - Finding best hyperparameters with cv

In [20]:
n_estimators = [100, 200, 250, 500]
learning_rate = [0.01,0.05,0.1,0.2]
random_state = [2,3,4]
reg_lambda = [0.1,1,10]
gamma = [0.1,1,10]
max_depth = [5, 8, 10, 12, 15]

In [21]:
param_grid = {'n_estimators' : n_estimators, 'random_state' : random_state,
              'gamma' : gamma, 'max_depth' : max_depth, 'learning_rate' : learning_rate, 'reg_lambda' : reg_lambda}

In [22]:
rf_grid = GridSearchCV(estimator=Regressor, param_grid=param_grid, cv=5)
rf_grid.fit(X, Y)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1,
                                    enable_categorical=False, gamma=0,
                                    gpu_id=-1, importance_type=None,
                                    interaction_constraints='',
                                    learning_rate=0.300000012, max_delta_step=0,
                                    max_depth=6, min_child_weight=1,
                                    missing=nan, monotone_constraints='()',
                                    n_estimators=100, n_jobs=4,
                                    num_parallel_tree=1, predictor='auto',
                                    random_state=0, reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, subsample=1,
                                    tree_method='exact', vali

In [23]:
rf_grid.best_params_

{'gamma': 10,
 'learning_rate': 0.1,
 'max_depth': 15,
 'n_estimators': 500,
 'random_state': 2,
 'reg_lambda': 10}

## Select From Model - Geothermal Gradient  

In [103]:
model = SelectFromModel(Regressor, prefit=True)
X_new = model.transform(X)

C:\Users\mathe\anaconda3\envs\Teste_Tensor\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [104]:
X_new.shape

(107, 45)

In [37]:
Regressor_new = XGBRegressor(gamma=10,
 learning_rate=0.05, max_depth=12, n_estimators=100, random_state=2, reg_lambda=1)
Regressor_new.fit(X_new, Y)
predict_new = Regressor_new.predict(X_new)

In [38]:
k = 5
kf = KFold(n_splits=k)
 
result = cross_val_score(Regressor_new , X_new, Y, cv = kf)
 
print("Avg accuracy: {}".format(result.mean()))

Avg accuracy: 0.20434351301806436


In [39]:
k = 5
kf = KFold(n_splits=k)
 
result = cross_val_score(Regressor_new , X_new, Y, cv = kf, scoring='neg_mean_absolute_error')
 
print("Avg accuracy: {}".format(result.mean()))

Avg accuracy: -3.6147668394166814


In [40]:
k = 5
kf = KFold(n_splits=k)
 
result = cross_val_score(Regressor_new , X_new, Y, cv = kf, scoring='neg_mean_squared_error')
 
print("Avg accuracy: {}".format(result.mean()))
rmse = math.sqrt(-result.mean())
rmse

Avg accuracy: -22.83006164205172


4.778081376666969

### Grid search for new model 

In [37]:
rf_grid = GridSearchCV(estimator=Regressor_new, param_grid=param_grid, cv=5)
rf_grid.fit(X_new, Y)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1,
                                    enable_categorical=False, gamma=0,
                                    gpu_id=-1, importance_type=None,
                                    interaction_constraints='',
                                    learning_rate=0.300000012, max_delta_step=0,
                                    max_depth=6, min_child_weight=1,
                                    missing=nan, monotone_constraints='()',
                                    n_estimators=100, n_jobs=4,
                                    num_parallel_tree=1, predictor='auto',
                                    random_state=0, reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, subsample=1,
                                    tree_method='exact', vali

In [38]:
rf_grid.best_params_

{'gamma': 10,
 'learning_rate': 0.05,
 'max_depth': 12,
 'n_estimators': 100,
 'random_state': 2,
 'reg_lambda': 1}

## Heat flow density

In [68]:
selected_rows = dataset[~dataset["Grad_Geot"].isnull()]

In [69]:
XX = pd.DataFrame((selected_rows.drop(["OBJECTID", "gridcode", "Shape_Length", "Shape_Area", "HFD", "Grad_Geot", "Count of Points"], axis=1)))
YY = (selected_rows["HFD"])

In [70]:
YY_mean, YY_std = mean(YY), std(YY)
# identify outliers
cut_off = YY_std * 3
lower, upper = YY_mean - cut_off, YY_mean + cut_off
# identify outliers
outliers = [x for x in YY if x < lower or x > upper]
print('Identified outliers: %d' % len(outliers))
# remove outliers
outliers_removed = [x for x in YY if x >= lower and x <= upper]
print('Non-outlier observations: %d' % len(outliers_removed))
print(outliers)

Identified outliers: 1
Non-outlier observations: 108
[183.84]


In [71]:
selected_rows = dataset[~dataset["HFD"].isnull()]
FinalData = selected_rows[~selected_rows['HFD'].isin(outliers)]

In [82]:
XX = pd.DataFrame((FinalData.drop(["OBJECTID", "gridcode", "Shape_Length", "Shape_Area", "HFD", "Grad_Geot", "Count of Points"], axis=1)))
YY = (FinalData["HFD"])

In [83]:
YY.shape

(108,)

In [84]:
YY.describe()

count    108.000000
mean      62.304904
std       18.095066
min       25.000000
25%       50.441450
50%       60.916100
75%       72.304250
max      116.650000
Name: HFD, dtype: float64

In [85]:
Regressor2 = XGBRegressor(gamma=10,
 learning_rate=0.1, max_depth=5, n_estimators=200, random_state=2, reg_lambda=0.1)
Regressor2.fit(XX, YY)
pred_Regressor = Regressor2.predict(XX)

In [86]:
k = 5
kf = KFold(n_splits=k)
result = cross_val_score(Regressor2 , XX, YY, cv = kf)
print (result.mean())

0.170151417830123


In [87]:
k = 5
kf = KFold(n_splits=k)
result = cross_val_score(Regressor2 , XX, YY, cv = kf, scoring='neg_mean_absolute_error')
print (result.mean())

-12.268504027405232


In [88]:
k = 5
kf = KFold(n_splits=k)
result = cross_val_score(Regressor2 , XX, YY, cv = kf, scoring='neg_mean_squared_error')
print (result.mean())
rmse = math.sqrt(-result.mean())
rmse

-246.84759273469007


15.711384176280907

In [56]:
rf_grid = GridSearchCV(estimator=Regressor2, param_grid=param_grid, cv=5)
rf_grid.fit(XX, YY)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1,
                                    enable_categorical=False, gamma=0,
                                    gpu_id=-1, importance_type=None,
                                    interaction_constraints='',
                                    learning_rate=0.300000012, max_delta_step=0,
                                    max_depth=6, min_child_weight=1,
                                    missing=nan, monotone_constraints='()',
                                    n_estimators=100, n_jobs=4,
                                    num_parallel_tree=1, predictor='auto',
                                    random_state=0, reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, subsample=1,
                                    tree_method='exact', vali

In [57]:
rf_grid.best_params_

{'gamma': 10,
 'learning_rate': 0.1,
 'max_depth': 5,
 'n_estimators': 200,
 'random_state': 2,
 'reg_lambda': 0.1}

### Select from model

In [106]:
model2 = SelectFromModel(Regressor2, prefit=True)
X_new2 = model2.transform(XX)

C:\Users\mathe\anaconda3\envs\Teste_Tensor\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [108]:
X_new2.shape

(108, 46)

In [90]:
Regressor_new2 = XGBRegressor(gamma=0.1,
 learning_rate=0.01, max_depth=12, n_estimators=500, random_state=2, reg_lambda=0.1)
Regressor_new2.fit(X_new2, YY)
predict_new2 = Regressor_new2.predict(X_new2)

In [91]:
k = 5
kf2 = KFold(n_splits=k)
 
result = cross_val_score(Regressor_new2 , X_new2, YY, cv = kf2)
 
print("Avg accuracy: {}".format(result.mean()))

Avg accuracy: 0.22028866093380697


In [72]:
k = 5
kf2 = KFold(n_splits=k)
 
result = cross_val_score(Regressor_new2 , X_new2, YY, cv = kf2, scoring='neg_mean_absolute_error')
 
print("Avg accuracy: {}".format(result.mean()))

Avg accuracy: -12.02276277140476


In [73]:
k = 5
kf2 = KFold(n_splits=k)
 
result = cross_val_score(Regressor_new2 , X_new2, YY, cv = kf2, scoring='neg_mean_squared_error')
 
print("Avg accuracy: {}".format(result.mean()))
rmse = math.sqrt(-result.mean())
rmse

Avg accuracy: -234.28529289604006


15.306380790246925

In [68]:
rf_grid = GridSearchCV(estimator=Regressor_new2, param_grid=param_grid, cv=5)
rf_grid.fit(X_new2, YY)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1,
                                    enable_categorical=False, gamma=0,
                                    gpu_id=-1, importance_type=None,
                                    interaction_constraints='',
                                    learning_rate=0.300000012, max_delta_step=0,
                                    max_depth=6, min_child_weight=1,
                                    missing=nan, monotone_constraints='()',
                                    n_estimators=100, n_jobs=4,
                                    num_parallel_tree=1, predictor='auto',
                                    random_state=0, reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, subsample=1,
                                    tree_method='exact', vali

In [69]:
rf_grid.best_params_

{'gamma': 0.1,
 'learning_rate': 0.01,
 'max_depth': 12,
 'n_estimators': 500,
 'random_state': 2,
 'reg_lambda': 0.1}

## Predict Full Data

In [98]:
full_data = pd.read_csv("Table_5x5_Final.csv")
X_full = (full_data.drop(["OBJECTID", "gridcode", "Shape_Length", "Shape_Area", "HFD", "Grad_Geot", "Count of Points"], axis=1))
Y_full = (full_data["HFD"])
X_full.shape

(3830, 241)

In [99]:
X_full = model2.transform(X_full)

C:\Users\mathe\anaconda3\envs\Teste_Tensor\lib\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [100]:
predict_full = Regressor_new2.predict(X_full)
predict_full

array([60.884758, 66.99465 , 77.84086 , ..., 70.98367 , 47.21288 ,
       78.52236 ], dtype=float32)

In [101]:
predict_full = pd.DataFrame(predict_full)
predict_full.describe()

,0
count,3830.000000
mean,63.905807
std,10.969373
min,24.987192
25%,57.619921
50%,63.930004
75%,70.023767
max,115.122223


In [102]:
prediction = pd.DataFrame(predict_full).to_excel("XGB_5x5_Selected_outliersout_hfd.xlsx")